### Ladi_ai_RAG

## RAG Setting

### Json_Loader 초거대AI 헬스케어 질의응답 데이터

https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&dataSetSn=71762

### file preprocessing

#### file load

In [1]:
import os
import json 
from glob import glob
from langchain.schema import Document
from tqdm import tqdm  # tqdm 라이브러리 임포트

# 기본 경로 설정
base_dir = "data/120.초거대AI 사전학습용 헬스케어 질의응답 데이터/healthcare_data_ladi"

# 질병 카테고리 및 이름, 의도 초기화
disease_categories = [folder for folder in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, folder))]
disease_names = []
disease_intentions = []

for folder in disease_categories:
    try:
        names = os.listdir(os.path.join(base_dir, folder))
        # .DS_Store 파일 필터링
        names = [name for name in names if name != '.DS_Store']
        disease_names.append(names)
        
        for disease_name in names:
            intentions = os.listdir(os.path.join(base_dir, folder, disease_name))
            # .DS_Store 파일 필터링
            intentions = [intention for intention in intentions if intention != '.DS_Store']
            disease_intentions.append(intentions)
    except Exception as e:
        print(f"{folder} 폴더 처리 중 오류 발생: {e}")

print("질병 카테고리:", disease_categories)
print("질병 이름:", disease_names)
print("질병 의도:", disease_intentions)
for category, names, intentions in zip(disease_categories, disease_names, disease_intentions):
    for name in names:
        # 모든 의도를 가져오기 위해 intentions의 인덱스를 사용하지 않음
        intention_list = intentions  # 모든 의도를 포함
        print(f"카테고리: {category}, 이름: {name}, 의도: {intention_list}")

질병 카테고리: ['피부질환', '근골격질환', '유전질환', '뇌신경정신질환', '여성질환', '소화기질환', '호흡기질환', '귀코목질환', '순환기질환', '감염성질환']
질병 이름: [['비듬', '기저귀 피부염', '약물 발진', '여드름', '오타 모반', '농가진', '사면발이증', '옻 중독', '모공성 각화증', '콜린성 두드러기', '내향성 손발톱', '무모증', '사마귀', '선천성 색소결핍증', '섬유종', '수부 습진', '두부 백선', '건선', '모반', '아토피성 피부염', '안면 홍조증', '표피낭종', '옴', '기미', '안면홍조', '탈모증', '땀띠', '피부건조증', '광선 각화증', '돌발성 발진', '티눈', '백반증', '대상포진', '간찰성 홍반', '두드러기', '소양증', '주근깨', '피부염', '스티븐 존슨 증후군', '켈로이드', '피부근육염', '국소 다한증', '동전모양 습진', '다모증', '대상포진 후 신경통', '화염상 모반', '피부섬유육종', '동상', '탈락 피부염', '베체트 병', '탄저병', '봉와직염', '지방종', '피부 농양', '단순 포진', '만성 두드러기', '한포진', '태열', '비립종', '쥐젖', '욕창', '슈도모나스 감염', '단독', '흉터', '무좀', '한랭 두드러기', '어린선', '어루러기', '지루성 피부염', '한센병', '편평 태선', '습진', '만성 단순 태선', '모낭염', '접촉성 피부염'], ['아킬레스 건 파열', '건초염', '관절염', '결절종', '염좌', '십자 인대 손상', '지체장애', '통풍', '요추 추간판 탈출증', '강직성 척추염', '결정종', '외반슬', '선천성 편평발', '아킬레스 건염', '골관절염', '류마티스 관절염', '내반슬', '무지외반증', '구루병', '근감소증', '통풍성 관절염', '요추관 협착증', '소아마비', '족저 근막염', '골감소증', '좌골신경통', '골수염', '골육종', '척추측만증

In [30]:
import json
import os

structured_data = []  # 구조화된 데이터를 저장할 리스트
output_dir = 'structured_datas'  # 저장할 폴더 경로

# 저장할 폴더가 없으면 생성
os.makedirs(output_dir, exist_ok=True)

# 각 폴더에서 json 파일 처리
for category in disease_categories:
    # 현재 카테고리에 해당하는 disease_names 인덱스 찾기
    category_index = disease_categories.index(category)
    # 현재 카테고리의 disease_names 가져오기 
    current_names = disease_names[category_index]
    # 현재 카테고리의 intentions 가져오기
    current_intentions = disease_intentions[category_index]

    for name in tqdm(current_names, desc=f"{category} 카테고리 처리 중"):  # tqdm을 사용하여 진행률 표시
        # 현재 카테고리의 의도만 사용
        intentions = set(current_intentions)
        for intention in intentions:
            intention_path = os.path.join(base_dir, category, name, intention)
            json_files = glob(os.path.join(intention_path, "*.json"))  # 의도에 해당하는 폴더에서 json 파일 검색
            
            # 각 json 파일의 num_of_words를 기준으로 정렬하여 상위 3개 파일 선택
            json_files_with_words = []
            for json_file in json_files:
                try:
                    with open(json_file, 'r', encoding='utf-8') as f:
                        data = json.load(f)
                        num_of_words = data.get('num_of_words', 0)
                        json_files_with_words.append((json_file, num_of_words))
                except json.JSONDecodeError as e:
                    print(f"JSON 파일 파싱 중 오류 발생 - {json_file}: {e}")
                except Exception as e:
                    print(f"파일 처리 중 오류 발생 - {json_file}: {e}")

            # num_of_words 기준으로 정렬 후 상위 3개 파일 선택
            top_json_files = sorted(json_files_with_words, key=lambda x: x[1], reverse=True)[:3]

            for json_file, _ in top_json_files:
                try:
                    with open(json_file, 'r', encoding='utf-8') as f:
                        data = json.load(f)
                        
                        # 필요한 메타데이터만 구성
                        metadata = {
                            "category": data.get('disease_category'),
                            "id": data.get('fileName'),
                            "name": data.get('disease_name', {}).get('kor'),
                            "intention": intention  # 현재 의도를 topic으로 설정
                        }

                        # 콘텐츠 구성 (answer 필드를 하나의 문자열로 합침)
                        content = f"{data['answer']['intro']} {data['answer']['body']} {data['answer']['conclusion']}"

                        # 구조화된 데이터 추가
                        structured_data.append({
                            "metadata": metadata,
                            "content": content
                        })
                        
                except json.JSONDecodeError as e:
                    print(f"JSON 파일 파싱 중 오류 발생 - {json_file}: {e}")
                except Exception as e:
                    print(f"파일 처리 중 오류 발생 - {json_file}: {e}")

# 결과 출력
print(f"총 {len(structured_data)}개의 구조화된 데이터를 로드했습니다.")

# 구조화된 데이터를 json 파일로 저장
for i, data in enumerate(structured_data, start=1):
    output_file = os.path.join(output_dir, f'structured_data_{i}.json')
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

# 저장한 json 파일에서 데이터 불러오기
loaded_data = []
for i in range(1, len(structured_data) + 1):
    output_file = os.path.join(output_dir, f'structured_data_{i}.json')
    with open(output_file, 'r', encoding='utf-8') as f:
        loaded_data.append(json.load(f))

# 첫 번째 구조화된 데이터의 메타데이터와 콘텐츠 출력 (확인용)
if loaded_data:
    print("\n첫 번째 구조화된 데이터 메타데이터:")
    print(loaded_data[0]['metadata'])
    print("\n첫 번째 구조화된 데이터 콘텐츠:")
    print(loaded_data[0]['content'])

감염성질환 카테고리 처리 중: 100%|██████████| 35/35 [00:10<00:00,  3.36it/s]


총 5206개의 구조화된 데이터를 로드했습니다.

첫 번째 구조화된 데이터 메타데이터:
{'category': '피부질환', 'id': 'HC-A-02827781', 'name': '비듬', 'intention': '식이, 생활'}

첫 번째 구조화된 데이터 콘텐츠:
비듬은 지루성 피부염의 일종으로, 두피와 모발에서 피지 분비가 과도하게 일어나거나 피부 각질이 과도하게 쌓이는 것을 의미합니다. 지루성 피부염의 원인은 호르몬 변화나 유전적인 요인, 스트레스 등이 될 수 있습니다. 비듬은 두피의 가려움증을 동반하며, 머리에 기름기가 많고 냄새가 나는 특징이 있습니다. 또한 두피가 자주 건조해지고 머리털이 빠질 수 있습니다. 비듬을 치료하기 위해 샴푸에 의한 자극을 피하는 것이 중요합니다. 머리를 세발할 때는 약산성의 제품을 선택하고, 두피를 깨끗하게 유지하기 위해 하루에 한 번 머리를 잘 감는 것이 좋습니다. 또한, 샴푸 후에는 두피에 남은 세균과 곰팡이를 제거하기 위해 가볍게 샴푸를 흔들어 씻고 깨끗한 물로 헹궈야 합니다. 기름기가 많고 냄새가 심한 경우에는 기름기를 잘 흡수하는 샴푸나 비듬 억제제를 사용할 수 있습니다. 비듬은 완전히 치료되는 것은 어렵지만 일상적인 조치와 적절한 샴푸와 린스를 사용하여 증상을 완화시킬 수 있습니다. 의사의 처방에 따라 적절한 샴푸나 린스를 사용하는 것이 좋습니다. 특히 기름기와 냄새가 심한 경우에는 기름기를 잘 흡수하는 샴푸나 비듬 억제제를 사용하면 증상을 완화할 수 있습니다.


In [5]:
print(len(structured_data))
print(type(structured_data))
print(structured_data[2340]['metadata'])
print(structured_data[2340]['content'])

NameError: name 'structured_data' is not defined

In [33]:
print(structured_data[0])


{'metadata': {'category': '피부질환', 'id': 'HC-A-02827781', 'name': '비듬', 'intention': '식이, 생활'}, 'content': '비듬은 지루성 피부염의 일종으로, 두피와 모발에서 피지 분비가 과도하게 일어나거나 피부 각질이 과도하게 쌓이는 것을 의미합니다. 지루성 피부염의 원인은 호르몬 변화나 유전적인 요인, 스트레스 등이 될 수 있습니다. 비듬은 두피의 가려움증을 동반하며, 머리에 기름기가 많고 냄새가 나는 특징이 있습니다. 또한 두피가 자주 건조해지고 머리털이 빠질 수 있습니다. 비듬을 치료하기 위해 샴푸에 의한 자극을 피하는 것이 중요합니다. 머리를 세발할 때는 약산성의 제품을 선택하고, 두피를 깨끗하게 유지하기 위해 하루에 한 번 머리를 잘 감는 것이 좋습니다. 또한, 샴푸 후에는 두피에 남은 세균과 곰팡이를 제거하기 위해 가볍게 샴푸를 흔들어 씻고 깨끗한 물로 헹궈야 합니다. 기름기가 많고 냄새가 심한 경우에는 기름기를 잘 흡수하는 샴푸나 비듬 억제제를 사용할 수 있습니다. 비듬은 완전히 치료되는 것은 어렵지만 일상적인 조치와 적절한 샴푸와 린스를 사용하여 증상을 완화시킬 수 있습니다. 의사의 처방에 따라 적절한 샴푸나 린스를 사용하는 것이 좋습니다. 특히 기름기와 냄새가 심한 경우에는 기름기를 잘 흡수하는 샴푸나 비듬 억제제를 사용하면 증상을 완화할 수 있습니다.'}


#### file load in local

In [4]:
import os
import json
loaded_data = []
output_dir = 'structured_datas'
for i in range(1, len(os.listdir(output_dir)) + 1):
    output_file = os.path.join(output_dir, f'structured_data_{i}.json')
    with open(output_file, 'r', encoding='utf-8') as f:
        loaded_data.append(json.load(f))

# 첫 번째 구조화된 데이터의 메타데이터와 콘텐츠 출력 (확인용)
if loaded_data:
    print("\n첫 번째 구조화된 데이터 메타데이터:")
    print(loaded_data[0]['metadata'])
    print("\n첫 번째 구조화된 데이터 콘텐츠:")
    print(loaded_data[0]['content'])


첫 번째 구조화된 데이터 메타데이터:
{'category': '피부질환', 'id': 'HC-A-02827781', 'name': '비듬', 'intention': '식이, 생활'}

첫 번째 구조화된 데이터 콘텐츠:
비듬은 지루성 피부염의 일종으로, 두피와 모발에서 피지 분비가 과도하게 일어나거나 피부 각질이 과도하게 쌓이는 것을 의미합니다. 지루성 피부염의 원인은 호르몬 변화나 유전적인 요인, 스트레스 등이 될 수 있습니다. 비듬은 두피의 가려움증을 동반하며, 머리에 기름기가 많고 냄새가 나는 특징이 있습니다. 또한 두피가 자주 건조해지고 머리털이 빠질 수 있습니다. 비듬을 치료하기 위해 샴푸에 의한 자극을 피하는 것이 중요합니다. 머리를 세발할 때는 약산성의 제품을 선택하고, 두피를 깨끗하게 유지하기 위해 하루에 한 번 머리를 잘 감는 것이 좋습니다. 또한, 샴푸 후에는 두피에 남은 세균과 곰팡이를 제거하기 위해 가볍게 샴푸를 흔들어 씻고 깨끗한 물로 헹궈야 합니다. 기름기가 많고 냄새가 심한 경우에는 기름기를 잘 흡수하는 샴푸나 비듬 억제제를 사용할 수 있습니다. 비듬은 완전히 치료되는 것은 어렵지만 일상적인 조치와 적절한 샴푸와 린스를 사용하여 증상을 완화시킬 수 있습니다. 의사의 처방에 따라 적절한 샴푸나 린스를 사용하는 것이 좋습니다. 특히 기름기와 냄새가 심한 경우에는 기름기를 잘 흡수하는 샴푸나 비듬 억제제를 사용하면 증상을 완화할 수 있습니다.


### Retriever Test

In [45]:
result=rag.retriever.invoke("머리에 비듬이 많이 날 때는 어떻게 해야해???", top_k=5)
print(result)

AttributeError: 'RunnableSequence' object has no attribute 'retriever'

In [10]:
result=rag.retriever.search_with_score("머리에 비듬이 많이 날 때는 어떻게 해야해??", top_k=5)
print(result)

문서 점수 계산 중: 5206it [00:00, 229816.41it/s]

search_with_score 시간: 0.05초
(Document(metadata={'score': 0.4686391380119016}, page_content='비듬은 두피에서 발생하는 각질이 많이 나타나는 현상으로, 피부 건강의 일반적인 문제입니다. 비듬은 두피의 표면에 흰가루와 같은 현상이 보입니다. 이러한 현상은 각질 세포가 과도하게 증식하여 머리를 자주 감거나 청결하지 않을 때, 머리를 말리지 않았을 때, 스트레스, 환경오염, 두피 질환 등의 여러 가지 요인에 의해 발생할 수 있습니다. 비듬이 발생하는 가장 일반적인 원인은 두피의 각질이 과도하게 증식하는 것입니다. 이러한 현상은 스트레스, 환경오염, 두피의 건조, 샴푸나 린스에 함유된 특정 성분에 대한 알레르기 반응, 심한 샴푸 사용 등 다양한 원인으로 인해 발생할 수 있습니다. 또한 두피의 혈액순환이 감소하거나 두피 세포의 증식이 과도할 때도 비듬이 생길 수 있습니다. 비듬은 정확한 원인을 찾는 것이 중요하며, 전문가의 조언을 받아 적절한 치료 방법을 찾는 것이 좋습니다. 또한 두피를 깨끗하게 유지하고 적절한 샴푸를 사용하여 비듬이 발생하지 않도록 관리하는 것이 중요합니다.'), Document(metadata={'score': 0.28040515150160505}, page_content='비듬은 지루성 피부염의 일종으로, 두피와 모발에서 피지 분비가 과도하게 일어나거나 피부 각질이 과도하게 쌓이는 것을 의미합니다. 지루성 피부염의 원인은 호르몬 변화나 유전적인 요인, 스트레스 등이 될 수 있습니다. 비듬은 두피의 가려움증을 동반하며, 머리에 기름기가 많고 냄새가 나는 특징이 있습니다. 또한 두피가 자주 건조해지고 머리털이 빠질 수 있습니다. 비듬을 치료하기 위해 샴푸에 의한 자극을 피하는 것이 중요합니다. 머리를 세발할 때는 약산성의 제품을 선택하고, 두피를 깨끗하게 유지하기 위해 하루에 한 번 머리를 잘 감는 것이 좋습니다. 또한, 샴푸 후에는 두피에 남은 세균과 곰팡이를 제거하기 위해 가볍

In [12]:
# 1. 벡터 DB에 저장된 문서 확인
def check_documents(retriever):
    print("저장된 문서 목록:")
    for doc in retriever.docs:
        print(f"ID: {doc.metadata.get('id')}, Topic: {doc.metadata.get('intention')}")
        print(f"Content: {doc.page_content[:100]}...")  # 내용의 일부만 출력
        print("-" * 50)

# 2. 리트리버의 검색 결과 확인
def test_retriever_query(retriever, query):
    print(f"'{query}'에 대한 검색 결과:")
    results = retriever.search_with_score(query, top_k=5)
    for i, doc in enumerate(results):
        print(f"Result {i + 1}:")
        print(f"ID: {doc.metadata.get('id')}, intention: {doc.metadata.get('intention')}")
        print(f"Content: {doc.page_content[:100]}...")  # 내용의 일부만 출력
        print(f"Score: {doc.metadata.get('score')}")
        print("-" * 50)

# 사용 예시
# 1. 리트리버의 문서 확인
#check_documents(rag.retriever)

# 2. 리트리버에 간 건강에 대한 질문 테스트
test_retriever_query(rag_retriever, "비듬 예방을 위해 예방 및 검진 조치까지 도와줘")

'비듬 예방을 위해 예방 및 검진 조치까지 도와줘'에 대한 검색 결과:


문서 점수 계산 중: 5206it [00:00, 229639.97it/s]

search_with_score 시간: 0.05초
Result 1:
ID: None, intention: None
Content: 비듬은 많은 사람들에게 불편을 주는 피부 문제입니다. 약물 치료는 이러한 증상을 관리하기 위한 일반적인 선택지 중 하나입니다. 가장 일반적으로 사용되는 약물 중 하나는 피록톤 올라...
Score: 0.44009122640597653
--------------------------------------------------
Result 2:
ID: None, intention: None
Content: 비듬 치료에는 다양한 약물이 사용됩니다. 특히 비듬 치료에는 약용 삼푸(비듬을 예방하고 개선하는 약물)와 비듬 치료제(비듬을 완화시키는 약물)의 사용이 중요합니다. 1. 비듬 예방...
Score: 0.3870315282292353
--------------------------------------------------
Result 3:
ID: None, intention: None
Content: 비듬은 피부 각질과 과다한 유분 분비로 인해 발생하는 피부 질환입니다. 이러한 비듬을 치료하기 위해 다양한 방법들이 있습니다. 가장 흔한 치료 방법 중 하나는 매일 샴푸를 사용하는...
Score: 0.08061319051338085
--------------------------------------------------
Result 4:
ID: None, intention: None
Content: 비듬은 두피의 기름기와 각질로 인해 생기는 질환이며, 주로 머리 감을 때 발생하는 흔한 문제입니다. 비듬의 치료에는 몇 가지 방법이 있습니다. 첫째로, 머리를 깨끗하게 유지하고 깨...
Score: 0.020973216574321604
--------------------------------------------------
Result 5:
ID: None, intention: None
Content: 비듬은 두피에서 발생하는 흔한 피부

In [12]:
print(rag.chain)
rag=rag.chain


first={
  context: RunnableLambda(itemgetter('context')),
  question: RunnableLambda(itemgetter('question')),
  age: RunnableLambda(itemgetter('user.age')),
  gender: RunnableLambda(itemgetter('user.gender')),
  job: RunnableLambda(itemgetter('user.job')),
  goal: RunnableLambda(itemgetter('goal')),
  habitCategory: RunnableLambda(itemgetter('habitCategory')),
  actions: RunnableLambda(itemgetter('actions'))
} middle=[StructuredPrompt(input_variables=['actions ?? defaultValue', 'context', 'goal ?? defaultValue', 'habitCategory ?? defaultValue', 'question', 'user.age ?? defaultValue', 'user.gender ?? defaultValue', 'user.job ?? defaultValue'], metadata={'lc_hub_owner': 'minuum', 'lc_hub_repo': 'ladi-241105-new', 'lc_hub_commit_hash': '3abcda354a7aa0ffcd7ac6ab15d369ebd20136265071e922e414616f0dbe4fd3'}, messages=[SystemMessage(content='라디는 자기계발의 복잡함과 동기 부족 문제를 해결하기 위해, AI 맞춤형 루틴 추천, 데이터 시각화, 커뮤니티 지원 기능을 통해 사용자가 자신의 목표를 달성할 수 있도록 돕는 것을 목표로 합니다.\n\n아래는 사용자가 제공한 정보입니다.\n이 데이터를 참고하여 구체적이고 명확한

- API DOCS : http://43.202.173.71:3000/
- TASK : 루틴 추천
- 루틴 관련 데이터 저장(vectorDB) -> 검색(Retrieve) -> LLM(gpt-4o)의 Context로 사용

### makeAction

In [5]:
from rag.base import RetrievalChain
from rag.json import JSONRetrievalChain
rag = JSONRetrievalChain(source_uri=["pass"], docs=["pass"])
rag=rag.create_chain(cache_mode='load', local_db="./cached_healthcare/", category="의료", mode='kiwi',func="makeaction")
rag_retriever=rag.retriever
rag=rag.chain


로컬에서 벡터 저장소 로드 중
kiwi_bm25_의료 캐시 경로: cache/kiwi_bm25_의료
cache/kiwi_bm25_의료 로드 중
load_local 시간: 0.11초


In [6]:
# 사용자 정보 정의
user_info = {
    "user_id": 123,
    "name": "이상하",
    "age": 23,
    "height": 168,
    "weight": 65,
    "workout_frequency": "1회 미만",
    "workout_location": "집",
    "gender": "남성",
    "job": "학생",
}

habit_info = {
    "category": "의료",  # category로 변경
    "goal": "탈모를 예방하고 진행 속도를 늦추고 싶어"
}

# create_chain의 키와 일치하도록 make_action_query 수정
make_action_query = {
    "context": rag_retriever,
    "question": habit_info.get("goal"),  # question은 goal로 사용
    "user_id": user_info.get("user_id"),
    "name": user_info.get("name"),
    "age": user_info.get("age"),
    "height": user_info.get("height"),
    "weight": user_info.get("weight"),
    "workout_frequency": user_info.get("workout_frequency"),
    "workout_location": user_info.get("workout_location"),
    "gender": user_info.get("gender"),
    "job": user_info.get("job"),
    "category": habit_info.get("category"),  # habitCategory를 category로 변경
    "goal": habit_info.get("goal")
}

print(make_action_query)

{'context': KiwiBM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x127c7c610>), 'question': '탈모를 예방하고 진행 속도를 늦추고 싶어', 'user_id': 123, 'name': '이상하', 'age': 23, 'height': 168, 'weight': 65, 'workout_frequency': '1회 미만', 'workout_location': '집', 'gender': '남성', 'job': '학생', 'category': '의료', 'goal': '탈모를 예방하고 진행 속도를 늦추고 싶어'}


In [7]:
print(type(rag))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [8]:
# make_action_query 함수 수정
def make_action(user_info: dict):
    try:
        result = rag.invoke(make_action_query,)
        return result
    except Exception as e:
        print(f"Error in make_action_query: {e}")
        return None

results = make_action(user_info)
print(results)

actions=[Action(action='아침에 두피 마사지'), Action(action='비타민D 보충제 섭취'), Action(action='오메가-3 섭취'), Action(action='녹차 한 잔 마시기'), Action(action='스트레스 관리 명상'), Action(action='충분한 수면 취하기'), Action(action='물 8잔 마시기'), Action(action='아연 보충제 섭취'), Action(action='비타민C 섭취'), Action(action='저녁에 두피 마사지'), Action(action='항산화 식품 섭취'), Action(action='주 1회 레이저 치료'), Action(action='모발 건강 식단 유지'), Action(action='주 1회 전문가 상담'), Action(action='모발 이식 고려'), Action(action='모발 전용 샴푸 사용'), Action(action='자외선 차단제 사용'), Action(action='모자 착용하기'), Action(action='주 1회 두피 스크럽'), Action(action='주 1회 발모제 사용')] category='의료' goal='탈모 예방 및 진행 속도 늦추기'


In [9]:
actions_dict = {"actions": [results.actions[i].action for i in range(len(results.actions))]}
print(actions_dict)  # 각 액션을 리스트 형태로 딕셔너리 안에 출력합니다.

{'actions': ['아침에 두피 마사지', '비타민D 보충제 섭취', '오메가-3 섭취', '녹차 한 잔 마시기', '스트레스 관리 명상', '충분한 수면 취하기', '물 8잔 마시기', '아연 보충제 섭취', '비타민C 섭취', '저녁에 두피 마사지', '항산화 식품 섭취', '주 1회 레이저 치료', '모발 건강 식단 유지', '주 1회 전문가 상담', '모발 이식 고려', '모발 전용 샴푸 사용', '자외선 차단제 사용', '모자 착용하기', '주 1회 두피 스크럽', '주 1회 발모제 사용']}


In [10]:
habit_info["actions"]=actions_dict
make_action_result_query = {
    "context": rag_retriever,
    "question": habit_info.get("goal"),  # question은 goal로 사용
    "user_id": user_info.get("user_id"),
    "name": user_info.get("name"),
    "age": user_info.get("age"),
    "height": user_info.get("height"),
    "weight": user_info.get("weight"),
    "workout_frequency": user_info.get("workout_frequency"),
    "workout_location": user_info.get("workout_location"),
    "gender": user_info.get("gender"),
    "job": user_info.get("job"),
    "category": habit_info.get("category"),  # habitCategory를 category로 변경
    "goal": habit_info.get("goal"),
    "actions": habit_info.get("actions", [])  # actions 추가
}
print(make_action_result_query)


{'context': KiwiBM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x127c7c610>), 'question': '탈모를 예방하고 진행 속도를 늦추고 싶어', 'user_id': 123, 'name': '이상하', 'age': 23, 'height': 168, 'weight': 65, 'workout_frequency': '1회 미만', 'workout_location': '집', 'gender': '남성', 'job': '학생', 'category': '의료', 'goal': '탈모를 예방하고 진행 속도를 늦추고 싶어', 'actions': {'actions': ['아침에 두피 마사지', '비타민D 보충제 섭취', '오메가-3 섭취', '녹차 한 잔 마시기', '스트레스 관리 명상', '충분한 수면 취하기', '물 8잔 마시기', '아연 보충제 섭취', '비타민C 섭취', '저녁에 두피 마사지', '항산화 식품 섭취', '주 1회 레이저 치료', '모발 건강 식단 유지', '주 1회 전문가 상담', '모발 이식 고려', '모발 전용 샴푸 사용', '자외선 차단제 사용', '모자 착용하기', '주 1회 두피 스크럽', '주 1회 발모제 사용']}}


### makeRoutine

In [12]:
from rag.base import RetrievalChain
from rag.json import JSONRetrievalChain
rag = JSONRetrievalChain(source_uri=["pass"], docs=["pass"])
rag=rag.create_chain(cache_mode='load', local_db="./cached_healthcare/", category="의료", mode='kiwi',func="makeroutine")
rag_retriever=rag.retriever
rag=rag.chain


로컬에서 벡터 저장소 로드 중
kiwi_bm25_의료 캐시 경로: cache/kiwi_bm25_의료
cache/kiwi_bm25_의료 로드 중
load_local 시간: 0.13초


In [13]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
# 문서 출력 도우미 함수
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )


# 모델 초기화
model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")

# 상위 3개의 문서 선택
compressor = CrossEncoderReranker(model=model, top_n=3)

# 문서 압축 검색기 초기화
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=rag_retriever
)

# 압축된 문서 검색
compressed_docs = compression_retriever.invoke(make_action_result_query.get("goal"))

# 문서 출력

pretty_print_docs(compressed_docs)


/Users/minu/Library/Caches/pypoetry/virtualenvs/langchain-kr-0guSvvrv-py3.11/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Document 1:

탈모증은 많은 사람들에게 심리적인 어려움을 초래할 수 있습니다. 탈모증을 숨기는 것은 많은 어려움을 겪는 사람들에게 도움을 줄 수 있습니다. 하지만, 우리는 다양한 방법을 활용하여 탈모증을 효과적으로 숨길 수 있습니다. 1. 헤어스타일 조정: 머리카락을 가리기 위해 헤어스타일을 조절하는 것은 다른 사람들에게 보이는 탈모 부위를 숨기고 자신감을 높일 수 있는 방법입니다. 위에서 언급했던 헤어스타일은 효과적으로 적용할 수 있으며, 자신감을 부여하고 헤어스타일에 따른 외모 자신감을 높일 수 있습니다. 2. 액세서리 활용: 액세서리를 사용하여 머리카락을 강조하고 탈모 부위를 가리는 것이 도움이 됩니다. 헤어 스타일뿐만 아니라 목걸이, 브로치, 반지 등 다양한 액세서리를 활용하여 머리카락을 강조하고 자신감을 높일 수 있습니다. 3. 액세서리 스타일링: 머리카락을 강조하는 액세서리를 활용하여 스타일링하는 방법을 찾을 수 있습니다. 머리에 스카프나 모자를 사용하여 머리카락을 가림으로써 탈모 부위를 가리고 자신감을 높일 수 있습니다. 4. 스타일링 제품 사용: 스타일링 제품을 사용하여 머리카락을 강조하고 탈모 부위를 가리는 데 도움을 줄 수 있습니다. 헤어 스타일링 제품을 사용하여 머리카락을 강조하고 자신감을 높일 수 있습니다. 우리는 모발을 건강하게 유지하기 위해 일상 생활에서 올바른 생활 습관을 유지하고 탈모 예방에 도움이 되는 보조 제품을 사용하는 것이 좋습니다. 탈모증을 감추기 위해 많은 노력을 기울이지만, 일상 생활에서 올바른 생활 습관을 유지하고 탈모 예방에 도움이 되는 제품을 사용하여 자신감을 회복하고 탈모를 예방할 수 있습니다.
----------------------------------------------------------------------------------------------------
Document 2:

탈모증은 많은 사람들에게 심리적으로도 많은 스트레스를 주는 문제입니다. 탈모증을 숨기는데는 여러 가지 방법이 

In [14]:

routine_info={
    "selected_actions": make_action_result_query.get("actions"),
    "start_time": "07:00",
    "repeat_days": ["월", "수", "금"]
}


In [15]:
# 먼저 compression_retriever로 문서 검색
# 모델 초기화
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")

# 상위 3개의 문서 선택
compressor = CrossEncoderReranker(model=model, top_n=3)

# 문서 압축 검색기 초기화
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=rag_retriever
)
compressed_docs = compression_retriever.invoke(make_action_result_query.get("goal"))


In [16]:

# make_routine_query 구성
make_routine_query = {
    "context": compressed_docs,  # compression_retriever 대신 compressed_docs 사용
    "question": make_action_result_query.get("goal"),
    "user_id": make_action_result_query.get("user_id"),
    "name": make_action_result_query.get("name"),
    "age": make_action_result_query.get("age"),
    "gender": make_action_result_query.get("gender"),
    "job": make_action_result_query.get("job"),
    "weight": make_action_result_query.get("weight"),
    "height": make_action_result_query.get("height"),
    "workout_frequency": make_action_result_query.get("workout_frequency"),
    "workout_location": make_action_result_query.get("workout_location"),
    "category": make_action_result_query.get("category"),
    "goal": make_action_result_query.get("goal"),
    "selected_actions": routine_info.get("selected_actions"),
    "start_time": routine_info.get("start_time"),
    "repeat_days": routine_info.get("repeat_days")
}

# rag 실행
try:
    result = rag.invoke(make_routine_query)
    print(result)
except Exception as e:
    print(f"Error occurred: {str(e)}")

subRoutine=[RoutineSubStep(emoji='🌅', routine='아침에 두피 마사지', secondDuration=300), RoutineSubStep(emoji='☀️', routine='비타민D 보충제 섭취', secondDuration=60), RoutineSubStep(emoji='🍵', routine='녹차 한 잔 마시기', secondDuration=600), RoutineSubStep(emoji='🧘\u200d♂️', routine='스트레스 관리 명상', secondDuration=900), RoutineSubStep(emoji='💧', routine='물 8잔 마시기', secondDuration=1800), RoutineSubStep(emoji='🌙', routine='저녁에 두피 마사지', secondDuration=300)]


In [21]:
subroutine_dict = {
    "subroutines": [
        {
            "emoji": sub_step.emoji,
            "routine": sub_step.routine,
            "secondDuration": sub_step.secondDuration
        } for sub_step in result.subRoutine  # result.subRoutine가 리스트라고 가정
    ]
}
print(subroutine_dict)  # 서브루틴을 딕셔너리 형태로 출력합니다.

{'subroutines': [{'emoji': '🌅', 'routine': '아침에 두피 마사지', 'secondDuration': 300}, {'emoji': '☀️', 'routine': '비타민D 보충제 섭취', 'secondDuration': 60}, {'emoji': '🍵', 'routine': '녹차 한 잔 마시기', 'secondDuration': 600}, {'emoji': '🧘\u200d♂️', 'routine': '스트레스 관리 명상', 'secondDuration': 900}, {'emoji': '💧', 'routine': '물 8잔 마시기', 'secondDuration': 1800}, {'emoji': '🌙', 'routine': '저녁에 두피 마사지', 'secondDuration': 300}]}


### API Code

In [23]:
from fastapi import FastAPI
import httpx
import nest_asyncio

nest_asyncio.apply()  # 이벤트 루프 문제 해결

app = FastAPI()

@app.post("/routine/")
async def create_routine(user_info: dict, question: str):

    
    # API 호출 예시
    async with httpx.AsyncClient() as client:
        api_response = await client.post("http://localhost:8000/routine", json={
            "greeting": result.greeting,
            "routines": [routine.dict() for routine in result.routines],  # RoutineStep을 dict로 변환
            "tips": result.tips
        })
    
    # API 응답의 원본 텍스트 출력
    print(api_response.text)  # 응답의 원본 텍스트 출력

    try:
        api_response_json = api_response.json()
    except ValueError:
        api_response_json = {"error": "Invalid JSON response", "response_text": api_response.text}

    return {
        "greeting": result.greeting,
        "routines": [routine.dict() for routine in result.routines],  # RoutineStep을 dict로 변환
        "tips": result.tips,
        "api_response": api_response_json  # API 응답 추가
    }

import uvicorn

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [5688]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [5688]


In [13]:

from fastapi.middleware.cors import CORSMiddleware

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)


### Json_Loader

In [ ]:
import json
from langchain.schema import Document
from pprint import pprint

# JSON 파일 로드 함수 정의
def load_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# JSON 데이터 로드
data = load_json_file("data/dummy_data.json")
\
# 필요한 정보 추출 및 Document 객체 생성
docs = [
    Document(
        page_content=item["context"],
        metadata={
            "id": item["id"],
            "question": item["question"],
            "age": item["age"],
            "gender": item["gender"],
            "Job": item["Job"],
            "Difficult": item["Difficult"],
            "goal": item["goal"]
        }
    )
    for item in data
]

# 결과 출력
pprint(docs)

### PDF_Loader

In [ ]:
from pdf_rag.pdf import PDFRetrievalChain

pdf_chain = PDFRetrievalChain(["data/세계보건기구(WHO) 신체활동 권장지침.pdf"])
pdf_chain.create_embedding()  # cached_embeddings 초기화
pdf = pdf_chain.create_chain()
pdf_retriever = pdf.retriever
pdf_chain = pdf.chain

In [ ]:
pdf_retriever.invoke("신체활동 권장지침에 대해 설명해줘")

In [ ]:
pdf_chain.invoke({"context":pdf_retriever,"question":"신체활동 권장지침에 대해 설명해줘"})


### Embedding and Vectorstore

In [25]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_upstage import UpstageEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings

# 1. 벡터 저장소 생성
passage_embeddings = UpstageEmbeddings(model="solar-embedding-1-large-query")


fs = LocalFileStore("./cache/")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    passage_embeddings, fs, namespace=passage_embeddings.model
)
vectorstore = FAISS(embedding_function=cached_embeddings)


### text_spliting and add_documents

In [ ]:


# 2. 문서 임베딩 및 저장
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)
vectorstore.add_documents(texts)


### 더미 데이터 생성

### generate one

In [ ]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import Optional

class SubRoutine(BaseModel):
    """운동 루틴을 추천합니다."""
    emoji: str
    routineDetail: str
    minuteDuration: str

llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0,response_format=SubRoutine)
response = llm.invoke("30분 동안 할 수 있는 운동 루틴을 추천해주세요.")
# structured_llm = llm.with_structured_output(
#     SubRoutine,
#     method="json_schema",
#     include_raw=False,
#     strict=True
# )

# response = structured_llm.invoke("30분 동안 할 수 있는 운동 루틴을 추천해주세요.")
import json

content = json.loads(response.content)
json_response = json.dumps(content, indent=2, ensure_ascii=False)
print(json_response)
print(type(json_response))


### LLM Answer

In [33]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import PromptTemplate

# 5. RAG 파이프라인 구축
prompt_template = PromptTemplate(
    input_variables=["context", "question", "user_id", "age", "gender", "height", "weight", "goal_id", "frequency_id", "location_id", "start_time", "end_time", "repeat_days"],
        template="""당신은 API 서버입니다.
{context}를 참고하여 사용자에 맞는 루틴을 추천해주세요.

사용자 정보:
- 사용자 ID: {user_id}
- 나이: {age}
- 성별: {gender} 
- 신장: {height}cm
- 체중: {weight}kg
- 운동 목표 ID: {goal_id}
- 운동 빈도 ID: {frequency_id}
- 선호 운동 장소 ID: {location_id}
- 운동 시작 시간: {start_time}
- 운동 종료 시간: {end_time}
- 반복 요일: {repeat_days}

질문: {question}
답변:
응답 형식
{{
  "subRoutine": {{
      "emoji": "이모지",
      "routineDetail": "루틴 설명", 
      "minuteDuration": "시간(분 단위)"
  }}
}}
"""
)

retriever = vectorstore.as_retriever()

class SubRoutine(BaseModel):
    """운동 루틴을 추천합니다."""
    emoji: str = Field(description="이모지")
    routineDetail: str = Field(description="루틴 설명")
    minuteDuration: str = Field(description="시간(분 단위)")

llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0.7)
parser = JsonOutputParser(pydantic_object=SubRoutine)
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

retrieved_docs = retriever.invoke("30분 동안 할 수 있는 운동 루틴을 추천해주세요.")


In [ ]:

# RAG 체인 생성
from operator import itemgetter
chain = (
    {
        "question":itemgetter("question"),
        "context":itemgetter("context"),
        "age":itemgetter("age"),
        "gender":itemgetter("gender"),
        "job":itemgetter("job"),
        "difficult":itemgetter("difficult"),
        "goal":itemgetter("goal")
    }
    | prompt_template
    | llm
    | parser
)

# 결과 생성
results = []
for i in range(5):

    result = chain.invoke(
        {"context":retrieved_docs,"question":"시험 공부를 위한 목표 달성 루틴 생성해줘",
         "age":itemgetter("age"),"gender":itemgetter("gender"),
         "job":itemgetter("job"),"difficult":itemgetter("difficult"),
         "goal":itemgetter("goal")}
    )
    results.append(result)

# 결과 출력
for i, result in enumerate(results, 1):
    print(f"Result {i}:")
    print(result)
    print()

In [43]:
import random
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage


# 프롬프트 템플릿 수정
prompt_template = PromptTemplate(
    input_variables=["context", "question", "age", "gender", "job", "difficult", "goal", "emojis"],
    template="""당신은 API 서버입니다.
{context}를 참고하여 사용자에 맞는 루틴을 추천해주세요.
사용자의 나이는 {age}입니다.
사용자의 성별은 {gender}입니다.
사용자의 직업은 {job}입니다.
사용자의 겪고 있는 어려움은 {difficult}입니다.
사용자의 목표는 {goal}입니다.

가능한 한 다양하고 창의적인 루틴을 생성해 주세요. 루틴의 내용에 맞는 적절한 이모지를 응답에 포함해주세요.

질문: {question}
답변:
응답 형식
{{
  "subRoutine": [
    {{
      "emoji": "이모지",
      "routineDetail": "루틴 설명",
      "minuteDuration": 시간(분 단위)
    }},
    ...
  ]
}}
"""
)

# LLM 설정 (온도 증가 및 타임아웃 설정)
llm = ChatOpenAI(model="gpt-4o", temperature=0.9, request_timeout=30)

# 파서 설정
class SubRoutineItem(BaseModel):
    emoji: str = Field(description="이모지")
    routineDetail: str = Field(description="루틴 설명")
    minuteDuration: int = Field(description="시간(분 단위)")

class SubRoutine(BaseModel):
    subRoutine: List[SubRoutineItem]

parser = JsonOutputParser(pydantic_object=SubRoutine)

# RAG 체인 생성
def create_chain():
    def chain(input_dict):
        prompt = prompt_template.format(
            context=input_dict["context"],
            question=input_dict["question"],
            age=input_dict["age"],
            gender=input_dict["gender"],
            job=input_dict["job"],
            difficult=input_dict["difficult"],
            goal=input_dict["goal"]
        )
        try:
            response = llm([HumanMessage(content=prompt)])
            if isinstance(response.content, str):
                parsed_response = parser.parse(response.content)
                return parsed_response
            else:
                print(f"Unexpected response type: {type(response.content)}")
                return None
        except Exception as e:
            print(f"Error in chain: {e}")
            return None
    return chain

chain = create_chain()

# 결과 생성
# 기존에 작성된 부분은 그대로 유지합니다.

# ...


In [ ]:

# 입력 데이터 리스트 생성
input_datas = [
    {
        "context": retrieved_docs,
        "question": "요즘 피곤해",
        "age": "20",
        "gender": "남성",
        "job": "학생",
        "difficult": "공부 집중력 부족",
        "goal": "시험에서 좋은 성적 받기"
    },
    {
        "context": retrieved_docs,
        "question": "공부에 집중이 안 돼",
        "age": "21",
        "gender": "남성",
        "job": "학생",
        "difficult": "시험 스트레스",
        "goal": "성적 향상"
    },
    {
        "context": retrieved_docs,
        "question": "시간 관리가 어려워",
        "age": "22",
        "gender": "남성",
        "job": "학생",
        "difficult": "시간 부족",
        "goal": "효율적인 공부"
    },
    {
        "context": retrieved_docs,
        "question": "시험 준비 어떻게 해야 할까?",
        "age": "23",
        "gender": "남성",
        "job": "학생",
        "difficult": "공부 방법 몰라",
        "goal": "시험 합격"
    },
    {
        "context": retrieved_docs,
        "question": "집중력이 떨어져",
        "age": "24",
        "gender": "남성",
        "job": "학생",
        "difficult": "집중력 부족",
        "goal": "공부 효율 향상"
    }
]

# 결과 생성
results = []
for i, input_data in enumerate(input_datas, 1):
    try:
        result = chain(input_data)
        if result:
            results.append(result)
    except Exception as e:
        print(f"Error at case {i}: {e}")
        continue

# 결과 출력
for i, result in enumerate(results, 1):
    print(f"Result {i}:")
    print(result)
    print()


In [ ]:

# 결과 출력
for i, result in enumerate(results, 1):
    print(f"Result {i} {input_datas[i-1]['question']}:")
    print(result)
    print()